In [1]:
import os
from bottle import route, run
from bottle import template
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
# Import library to display results
import matplotlib.pyplot as plt
from bottle import request


os.chdir("C:\Indira\Codes\BottleApp")

In [2]:
from bottle import static_file,get
_url = 'https://westus.api.cognitive.microsoft.com/vision/v1.0/analyze?'
_key = 'ddd4c6b08a344f8494939e814aa45339'
_maxNumRetries = 10  
@get("/static/img/<filepath:re:.*\.(jpg|png|gif|ico|svg)>")
def img(filepath):
    return static_file(filepath, root="static/images/work/")

 
@get("/static/css/<filepath:re:.*\.css>")
def css(filepath):
    return static_file(filepath, root="static/css/")

@get("/static/js/<filepath:re:.*\.js>")
def js(filepath):
    return static_file(filepath, root="static/js/")


In [4]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None
    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['message'] ) )

        break
        
    return result

In [5]:
def renderResultOnImage( result, img ):
   
    """Display the obtained results onto the input image"""

    R = int(result['color']['accentColor'][:2],16)
    G = int(result['color']['accentColor'][2:4],16)
    B = int(result['color']['accentColor'][4:],16) ;
    cv2.rectangle( img,(0,0), (img.shape[1], img.shape[0]), color = (R,G,B), thickness = 25 )

    if 'description' in result:
        desc = result['description']['captions'][0]['text']
        cv2.putText( img, desc, (200,70), cv2.FONT_HERSHEY_DUPLEX, 2, (73,25,180), 2 )
        print(desc)


In [11]:
@route('/edit')
def edit():
    info = []
    return template('simple.tpl', info)


@route('/vision')
def vision():
    return(template("vision.tpl"))

@route('/vision/info')
def vision(info):
    return(template("visionWImg.tpl",info))



In [12]:
@route('/upload', method = 'POST')
def upload():
    upload = request.files.get('upload')
    name, ext = os.path.splitext(upload.filename)
    if ext not in ('.png', '.jpg', '.jpeg'):
        return "File extension not allowed."

    save_path = "./tmp/upload"
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    file_path = "{path}/{file}".format(path=save_path, file=upload.filename)
    upload.save(file_path)
    
    # Use the uploaded file for processing 
    # URL direction to image
    # urlImage = 'http://bboceansuites.com/wp-content/uploads/2015/05/sunset-beach-chairs-stock-footage.jpg'
    
    pathToFileInDisk = file_path
    with open( pathToFileInDisk, 'rb' ) as f:
        data = f.read()
    
    # Computer Vision parameters
    params = { 'visualFeatures' : 'Color,Categories,Faces,Description'} 

    headers = dict()
    headers['Ocp-Apim-Subscription-Key'] = _key
    #headers['Content-Type'] = 'application/json' 
    headers['Content-Type'] = 'application/octet-stream'
    json = None
    #json = { 'url': urlImage } 
    #data = None
    result = processRequest( json, data, headers, params )
    if result is not None:
        # Load the original image, fetched from the URL
        data8uint = np.fromstring( data, np.uint8 ) # Convert string to an unsigned int array
        img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
        renderResultOnImage( result, img )
        cv2.imwrite(os.path.join('./tmp/upload', "vision_op2.jpg"),img)
        info = img
        return template('vision.tpl', img)  

In [ ]:
run(host='localhost', port=8080, debug=True)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [27/Mar/2017 11:00:42] "GET / HTTP/1.1" 404 720
127.0.0.1 - - [27/Mar/2017 11:00:45] "GET /edit HTTP/1.1" 200 2131
127.0.0.1 - - [27/Mar/2017 11:00:45] "GET /static/images/pfd-sprites.png HTTP/1.1" 404 778
127.0.0.1 - - [27/Mar/2017 11:00:46] "GET /edit HTTP/1.1" 200 2131
127.0.0.1 - - [27/Mar/2017 11:00:46] "GET /static/images/pfd-sprites.png HTTP/1.1" 404 778
127.0.0.1 - - [27/Mar/2017 11:00:47] "GET /vision HTTP/1.1" 200 1286
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda2\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\ProgramData\Anaconda2\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "<ipython-input-12-d9218ffefc10>", line 4, in upload
    name, ext = os.path.splitext(upload.filename)
AttributeError: 'NoneType' object has no attribute 'filename'


a close up of two giraffes near a tree


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda2\lib\site-packages\bottle.py", line 862, in _handle
    return route.call(**args)
  File "C:\ProgramData\Anaconda2\lib\site-packages\bottle.py", line 1740, in wrapper
    rv = callback(*a, **ka)
  File "<ipython-input-12-d9218ffefc10>", line 41, in upload
    return template('vision.tpl', img)
  File "C:\ProgramData\Anaconda2\lib\site-packages\bottle.py", line 3618, in template
    for dictarg in args[1:]: kwargs.update(dictarg)
ValueError: dictionary update sequence element #0 has length 534; 2 is required
127.0.0.1 - - [27/Mar/2017 11:01:11] "POST /upload HTTP/1.1" 500 1595


In [89]:
help(cv2.imwrite)


Help on built-in function imwrite:

imwrite(...)
    imwrite(filename, img[, params]) -> retval

